# Load the pretrained model

Load annotations without Images

In [ ]:
curl -X GET http://url:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token xxx' --output 'annotations.csv' 

Archive Images (on Server!)

In [ ]:
tar -czvf /home/alex/all_imgs_11_10_23.tar.gz

Download archive from server and extract it

In [ ]:
scp alex@foodsnapai.com:/home/alex/all_imgs_11_10_23.tar.gz allImgs.tar.gz
tar -xzvf allImgs.tar.gz -C /home/alex/allImgs_extracted/

Convert annotations and images and preprocess them

In [ ]:
%run 'convert_csv_to_mlflow_csv.py' annotations.csv

In [ ]:
%run 'convert_pascal_to_googlecsv.py' --merged_csv annotations_mlflow.csv

Now rename the file paths in the file to their actual location (find & replace)

In [7]:
!pwd
%run 'png_to_jpeg.py' annotations_mlflow_shuffled.csv

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 22
Ignored 6206 files because: Not a PNG file
Ignored 0 files because: Not found


In [ ]:

%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800

In [8]:

%run normalize_csv.py annotations_mlflow_shuffled.csv annotations_mlflow_shuffled_n.csv

Now you can start to load the model and train!

In [9]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

spec = model_spec.get('efficientdet_lite0')

# Load the dataset

In [14]:
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)
file_path = '/home/alex/tflite_model_maker_wsl2/annotations_mlflow_shuffled_n.csv'

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)

Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file '/home/alex/tflite_model_maker_wsl2/annotations_mlflow_shuffled_n.csv' exists.


# Train the model

In [15]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data, epochs=1)

96/96 [==============================] - 915s 366ms/step - det_loss: 1.5958 - cls_loss: 1.0141 - box_loss: 0.0116 - reg_l2_loss: 0.0652 - loss: 1.6610 - learning_rate: 0.0090 - gradient_norm: 1.1492 - val_det_loss: 1.4603 - val_cls_loss: 0.9380 - val_box_loss: 0.0104 - val_reg_l2_loss: 0.0652 - val_loss: 1.5255


Run on test data

In [ ]:
model.evaluate(test_data)

Export the model to tflite model

In [17]:
model.export(export_dir='models/new_model')

2023-11-12 23:24:49.461495: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.797 G  ops, equivalently 0.898 G  MACs


2023-11-12 23:24:56.250520: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-12 23:24:56.250560: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-12 23:24:56.250718: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpo36ottr0
2023-11-12 23:24:56.315435: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-12 23:24:56.315465: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpo36ottr0
2023-11-12 23:24:56.518408: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-12 23:24:57.654126: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpo36ottr0
2023-11-12 23:24:58.140511: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1889794

Estimated count of arithmetic ops: 1.797 G  ops, equivalently 0.898 G  MACs


Run inference script for visual output

In [21]:
%run do_inference.py --input_csv annotations_mlflow_shuffled_n.csv\
                       --model_url models/new_model/model.tflite --output_dir output_inference

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
  0%|          | 0/96 [00:00<?, ?it/s]

  1%|          | 1/96 [00:01<02:18,  1.46s/it]

Prediction saved to output_inference/prediction_ac0ed1b0-d9cac84833465320_IMG_20230809_130228_HDR.jpg


  2%|▏         | 2/96 [00:02<02:17,  1.47s/it]

Prediction saved to output_inference/prediction_a651e069-4888f0ad0981b6e0_20230907_120546.jpg


...

Prediction saved to output_inference/prediction_c76727c6-9cdd8edc90278f47_20230814_164734.jpg


 10%|█         | 10/96 [00:14<02:06,  1.47s/it]

Prediction saved to output_inference/prediction_8f886102-f32a646689eb11ac_9408F590-421E-467E-984E-B383F2F9F21A.jpeg


 10%|█         | 10/96 [00:16<02:18,  1.61s/it]


KeyboardInterrupt: 